In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# SNU PLANETA - forecasting PM2.5 24hours later

- best RMSE : 15.05

## 1. Data load

In [2]:
!gdown https://drive.google.com/u/0/uc?id=1BDLRo8NIVk9WEhhr5vW_C9n5zqckGLMH
!7z x '/content/4.______.zip' -odataset

Downloading...
From: https://drive.google.com/u/0/uc?id=1BDLRo8NIVk9WEhhr5vW_C9n5zqckGLMH
To: /content/4.______.zip
18.7MB [00:00, 115MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 18726405 bytes (18 MiB)

Extracting archive: /content/4.______.zip
--
Path = /content/4.______.zip
Type = zip
Physical Size = 18726405

  0%     21% 7 - train/train_input_aws.csv                                   42% 7 - train/train_input_aws.csv                                   66% 8 - train/train_input_pm25.csv                                   

In [164]:
%%capture
!pip install geopandas
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import folium
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt

import os

tf.random.set_seed(7)

In [379]:
data_dir = '/content/dataset'

# 훈련, 검증 데이터 경로 지정
train_input_aws_path = os.path.join(data_dir, 'train/train_input_aws.csv')
train_input_pm25_path = os.path.join(data_dir, 'train/train_input_pm25.csv')
train_output_path = os.path.join(data_dir, 'train/train_output_pm25.csv')

test_input_pm25_path = os.path.join(data_dir, 'test/test_input_pm25.csv')
test_input_aws_path = os.path.join(data_dir, 'test/test_input_aws.csv')
test_output_path = os.path.join(data_dir, 'test/sample_answer.csv')

# 지정한 경로에서 데이터 불러오기
# 훈련 데이터
train_input_aws_df = pd.read_csv(train_input_aws_path)
train_input_pm25_df = pd.read_csv(train_input_pm25_path)
train_output_df = pd.read_csv(train_output_path)

# 검증 데이터
test_input_aws_df = pd.read_csv(test_input_aws_path)
test_input_pm25_df = pd.read_csv(test_input_pm25_path)
submission = pd.read_csv(test_output_path)

# 장소 정보 불러오기
location_path = os.path.join(data_dir, 'train/locations.csv')
location_df = pd.read_csv(location_path)

## 2. Data Preprocess

In [380]:
# 장소카테고리별로 그룹
grouped_location_df = location_df.groupby('category')

# 장소를 aws, pm25로 나누어주기
aws_loc_df = grouped_location_df.get_group('aws').reset_index(drop=True)
pm25_loc_df = grouped_location_df.get_group('pm25').reset_index(drop=True)

In [381]:
# GeoDataFrame으로 변환
aws_loc_gdf = gpd.GeoDataFrame(aws_loc_df, geometry=gpd.points_from_xy(aws_loc_df.longitude, aws_loc_df.latitude))
pm25_loc_gdf = gpd.GeoDataFrame(pm25_loc_df, geometry=gpd.points_from_xy(pm25_loc_df.longitude, pm25_loc_df.latitude))

In [383]:
# 좌표계 변환
aws_loc_gdf.crs = {'init':'epsg:4326'}
pm25_loc_gdf.crs = {'init':'epsg:4326'}

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [384]:
# 미터 좌표계로 변환
aws_loc_gdf = aws_loc_gdf.to_crs({'init':'epsg:5179'})
pm25_loc_gdf = pm25_loc_gdf.to_crs({'init':'epsg:5179'})

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [385]:
aws_loc_df

,category,loc_code,loc_name,latitude,longitude,geometry
0,aws,400,강남,37.49820,127.08162,POINT (127.08162 37.49820)
1,aws,402,강동,37.55556,127.14498,POINT (127.14498 37.55556)
2,aws,424,강북,37.63972,127.02576,POINT (127.02576 37.63972)
3,aws,404,강서,37.57390,126.82953,POINT (126.82953 37.57390)
4,aws,509,관악,37.45284,126.95015,POINT (126.95015 37.45284)
5,aws,423,구로,37.49328,126.82629,POINT (126.82629 37.49328)
6,aws,417,금천,37.49328,126.82630,POINT (126.82630 37.49328)
7,aws,425,남현,37.46347,126.98154,POINT (126.98154 37.46347)
8,aws,407,노원,37.62186,127.09192,POINT (127.09192 37.62186)
9,aws,406,도봉,37.66612,127.02947,POINT (127.02947 37.66612)


In [386]:
# aws_loc_df 시각화
locations = aws_loc_df[['latitude', 'longitude']]
locationlist = locations.values.tolist()

center = locationlist[19]  # 용산 aws 를 중심으로 folium map
map = folium.Map(location=center, zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=aws_loc_df['loc_name'][point]).add_to(map)
map

In [387]:
# aws, pm25 좌표 딕셔너리 생성
def geo_var(df, dict):
  for i in df['loc_name']:
    x = df.loc[df.loc_name == i, "geometry"].squeeze()
    dict[i] = x
  return dict

aws_dict ={}
pm25_dict={}

aws_dict = geo_var(aws_loc_gdf, aws_dict)
pm25_dict = geo_var(pm25_loc_gdf, pm25_dict)

In [388]:
# pm25의 각 좌표와 가장 가까운 aws 좌표를 매칭
def nearest_distance(dict1, dict2):
  distance={}
  for t in list(dict1.items()):
    k1 = t[0]
    v1 = t[1]
    shortest = v1.distance(list(dict2.values())[0])
    for m in list(dict2.items()):
      k2 = m[0]
      v2 = m[1]
      if v1.distance(v2) < shortest:
        shortest = v1.distance(v2)
        index=k2
    distance[k1] = index
  return distance

pm25_nearest = nearest_distance(pm25_dict, aws_dict)

In [389]:
# 매칭한 장소 이름과 좌표를 데이터 프레임으로 변환
pm25_nearest = pd.DataFrame(list(pm25_nearest.items()), columns=['loc_name','aws_loc_name'])

In [390]:
# pm25_loc_df, aws_loc_df에 매칭한 장소를 merge

pm25_loc_df = pd.merge(pm25_loc_df, pm25_nearest, how = 'inner', on=['loc_name'])

aws_loc_df = aws_loc_df.rename(columns={'loc_name': 'aws_loc_name'})

In [392]:
# Merge (pm25_loc_df, aws_loc_df)
loc_df = pd.merge(pm25_loc_df, aws_loc_df, how='inner', on = ['aws_loc_name'])

# get id for every pm25 loc_code
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
loc_df['loc_id'] = lbl.fit_transform(loc_df['loc_code_x'])

In [393]:
# make dataframe with loc_id
pm25_loc_id = loc_df[['loc_code_x','loc_id']]
aws_loc_id = loc_df[['loc_code_y', 'loc_id']]

pm25_loc_id = pm25_loc_id.rename(columns={'loc_code_x':'loc_code'})
aws_loc_id = aws_loc_id.rename(columns={'loc_code_y':'loc_code'})

In [394]:
# first attempt, 별로임
'''
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
aws_loc_df['loc_id'] = lbl.fit_transform(aws_loc_df['loc_name'])
aws_loc_df.head()

loc_id_map = aws_loc_df['loc_name'].to_dict()
pm25_loc_df['loc_name'] = pm25_loc_df['loc_name'].map(lambda x: x[:-1])

loc_id_map[22] = '광진'
loc_id_map[23] = '동작'

loc_id_map = {v: k for k, v in loc_id_map.items()}
loc_id_map['종로'] = 10
loc_id_map['중'] = 12
loc_id_map['영등포'] = 18

print(loc_id_map)

pm25_loc_df['loc_id'] = pm25_loc_df['loc_name'].map(loc_id_map.get).astype(int)

aws = aws_loc_df.drop(['category', 'loc_name', 'latitude', 'longitude'], axis=1)
pm25 = pm25_loc_df.drop(['category', 'loc_name', 'latitude', 'longitude'], axis=1)

pm25_aws_loc = pd.merge(pm25, aws, how='inner', on=['loc_id'])
pm25_aws_loc.rename(columns={'loc_code_x':'pm25_loc_code', 'loc_code_y':'aws_loc_code'}, inplace=True)
pm25_aws_loc['loc_id'] = lbl.fit_transform(pm25_aws_loc['pm25_loc_code'])

pm25_loc_id = pm25_aws_loc.iloc[:,:2]
aws_loc_id = pm25_aws_loc.iloc[:,1:]

pm25_loc_id.rename(columns={'pm25_loc_code':'loc_code'}, inplace=True)
aws_loc_id.rename(columns={'aws_loc_code':'loc_code'}, inplace=True)
'''

"\nfrom sklearn.preprocessing import LabelEncoder\nlbl = LabelEncoder()\naws_loc_df['loc_id'] = lbl.fit_transform(aws_loc_df['loc_name'])\naws_loc_df.head()\n\nloc_id_map = aws_loc_df['loc_name'].to_dict()\npm25_loc_df['loc_name'] = pm25_loc_df['loc_name'].map(lambda x: x[:-1])\n\nloc_id_map[22] = '광진'\nloc_id_map[23] = '동작'\n\nloc_id_map = {v: k for k, v in loc_id_map.items()}\nloc_id_map['종로'] = 10\nloc_id_map['중'] = 12\nloc_id_map['영등포'] = 18\n\nprint(loc_id_map)\n\npm25_loc_df['loc_id'] = pm25_loc_df['loc_name'].map(loc_id_map.get).astype(int)\n\naws = aws_loc_df.drop(['category', 'loc_name', 'latitude', 'longitude'], axis=1)\npm25 = pm25_loc_df.drop(['category', 'loc_name', 'latitude', 'longitude'], axis=1)\n\npm25_aws_loc = pd.merge(pm25, aws, how='inner', on=['loc_id'])\npm25_aws_loc.rename(columns={'loc_code_x':'pm25_loc_code', 'loc_code_y':'aws_loc_code'}, inplace=True)\npm25_aws_loc['loc_id'] = lbl.fit_transform(pm25_aws_loc['pm25_loc_code'])\n\npm25_loc_id = pm25_aws_loc.ilo

In [395]:
# loc_id 정보를 매핑
train_input_pm25_df = pd.merge(train_input_pm25_df, pm25_loc_id, how='inner', on=['loc_code'])
test_input_pm25_df = pd.merge(test_input_pm25_df, pm25_loc_id, how='inner', on=['loc_code'])

train_input_aws_df = pd.merge(train_input_aws_df, aws_loc_id, how='inner', on=['loc_code'])
test_input_aws_df = pd.merge(test_input_aws_df, aws_loc_id, how='inner', on=['loc_code'])

# 필요 없는 정보는 제거
train_input_aws_df = train_input_aws_df.drop(['id', 'loc_code', 'humidity'], axis=1)
test_input_aws_df = test_input_aws_df.drop(['id', 'loc_code', 'humidity'], axis=1)

In [399]:
'''
train_input_pm25_df['datetime'] = pd.to_datetime(train_input_pm25_df['time'], format='%Y%m%d%H')
test_input_pm25_df['datetime'] = pd.to_datetime(test_input_pm25_df['time'], format='%Y%m%d%H')
train_input_aws_df['datetime'] = pd.to_datetime(train_input_aws_df['time'], format='%Y%m%d%H')
test_input_aws_df['datetime'] = pd.to_datetime(test_input_aws_df['time'], format='%Y%m%d%H')
'''

"\ntrain_input_pm25_df['datetime'] = pd.to_datetime(train_input_pm25_df['time'], format='%Y%m%d%H')\ntest_input_pm25_df['datetime'] = pd.to_datetime(test_input_pm25_df['time'], format='%Y%m%d%H')\ntrain_input_aws_df['datetime'] = pd.to_datetime(train_input_aws_df['time'], format='%Y%m%d%H')\ntest_input_aws_df['datetime'] = pd.to_datetime(test_input_aws_df['time'], format='%Y%m%d%H')\n"

In [400]:
'''
from datetime import datetime, timedelta
train_input_aws_df['datetime'] = train_input_aws_df['datetime'].apply(lambda x: x - timedelta(hours=24))
test_input_aws_df['datetime'] = test_input_aws_df['datetime'].apply(lambda x: x - timedelta(hours=24))

def to_integer(dt_time):
    return 1000000*dt_time.year + 10000*dt_time.month + 100*dt_time.day + dt_time.hour

train_input_aws_df['time'] = train_input_aws_df['datetime'].apply(lambda x: to_integer(x))
test_input_aws_df['time'] = test_input_aws_df['datetime'].apply(lambda x: to_integer(x))
'''

"\nfrom datetime import datetime, timedelta\ntrain_input_aws_df['datetime'] = train_input_aws_df['datetime'].apply(lambda x: x - timedelta(hours=24))\ntest_input_aws_df['datetime'] = test_input_aws_df['datetime'].apply(lambda x: x - timedelta(hours=24))\n\ndef to_integer(dt_time):\n    return 1000000*dt_time.year + 10000*dt_time.month + 100*dt_time.day + dt_time.hour\n\ntrain_input_aws_df['time'] = train_input_aws_df['datetime'].apply(lambda x: to_integer(x))\ntest_input_aws_df['time'] = test_input_aws_df['datetime'].apply(lambda x: to_integer(x))\n"

In [401]:
# 훈련 데이터셋, 테스트 데이터셋 생성
# pm25 데이터와 aws 데이터를 merge
train = pd.merge(train_input_pm25_df, train_input_aws_df, how='left', on=['loc_id', 'time'])
test = pd.merge(test_input_pm25_df, test_input_aws_df, how='left', on=['loc_id', 'time'])

In [402]:
train['datetime'] = pd.to_datetime(train['time'], format='%Y%m%d%H')
test['datetime'] = pd.to_datetime(test['time'], format='%Y%m%d%H')

# 결측치 보간
train = train.set_index('datetime').interpolate(method='nearest') # (1051200, 9) =>  1051200 = 24(시) *  * 365(일) * 5(년) * 24(관측소)
test = test.set_index('datetime').interpolate(method='nearest') # (1051200, 9) =>  1051200 = 24(시) *  * 365(일) * 5(년) * 24(관측소)

In [404]:
# 결측치 개수 확인
print(train.isnull().sum())
print(test.isnull().sum())

id                 0
time               0
loc_code           0
PM25              25
loc_id             0
temperature        0
wind_direction     0
wind_speed         0
precipitation      0
dtype: int64
id                0
time              0
loc_code          0
PM25              0
loc_id            0
temperature       0
wind_direction    0
wind_speed        0
precipitation     0
dtype: int64


In [405]:
# 결측치 채우기
#train['humidity'] = train['humidity'].fillna(train['humidity'].mean())
#test['humidity'] = test['humidity'].fillna(test['humidity'].mean())

train = train.fillna(7.0)

# train 데이터 길이 변수 생성: 향후 데이터 합치고 나눌 때 사용.
split = train.shape[0]

In [407]:
# 훈련, 검증 데이터셋 통합 total 데이터셋 생성
total = pd.concat([train, test])

In [408]:
wv = total.pop('wind_speed')
max_wv = wv.max()

# Convert to radians.
wd_rad = total.pop('wind_direction')*np.pi / 180

# Calculate the wind x and y components.
total['Wx'] = wv*np.cos(wd_rad)
total['Wy'] = wv*np.sin(wd_rad)

# Calculate the max wind x and y components.
total['max Wx'] = max_wv*np.cos(wd_rad)
total['max Wy'] = max_wv*np.sin(wd_rad)

In [409]:
# datetime 정보 변수 생성
total = total.reset_index()

import datetime
timestamp_s = total['datetime'].map(datetime.datetime.timestamp)

day = 24*60*60
#month = 30*day
year = (365.2425)*day

total['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
total['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
total['Month sin'] = np.sin(timestamp_s * (2 * np.pi / month))
total['Month cos'] = np.cos(timestamp_s * (2 * np.pi / month))
total['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
total['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [410]:
# 인덱스 set, 필요 없는 loc_id 제거
total = total.set_index('datetime')
total = total.drop(['loc_id'], axis=1)

In [413]:
# train, test split
train = total[:split][:]
test = total[split:][:]

In [414]:
def reshape_dataframe(df, loc_df):
  # reshape the dataframe into 3d array with indexing [loc_code, value, time-id]
  # result[loc_code, value, time-id] will give you the values of the 24 hours
  len_loc = len(loc_df)
  col_values = df.columns[3:]  # not going to use `id`, `time`, `loc_code`
  len_values = len(col_values)
  len_id = int(df['id'].max()+1)
  len_term = 24
  result = np.ndarray((len_loc, len_values, len_id, len_term))

  # group by loc_code and id
  grouped_df = df.groupby(['loc_code', 'id'])
  group_keys = list(grouped_df.groups.keys())
  group_keys.sort()

  for i_key, (loc_key, id_key) in enumerate(group_keys):
    i_loc = int(i_key/len_id)
    i_key = i_key%len_id
    cur_df = grouped_df.get_group((loc_key, id_key))
    for icol, col in enumerate(col_values):
      result[i_loc][icol][i_key] = cur_df[col].to_numpy()
  return result

In [415]:
# 시계열 데이터 훈련에 적합한 numpy array 형태로 변환
train = reshape_dataframe(train, pm25_loc_df)
test_df = reshape_dataframe(test, pm25_loc_df)

days = 1825
train_df = train[:,:,:int(days*0.8),:]
val_df = train[:,:,int(days*0.8):,:]

In [417]:
# 데이터 정규화
train_mean = train.mean()
train_std = train.std()

train = (train - train_mean) / train_std
train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [418]:
# 타겟 데이터 생성
train_output_df['datetime'] = pd.to_datetime(train_output_df['time'], format='%Y%m%d%H')
train_output_df = train_output_df.set_index('datetime').interpolate(method='time')

target = reshape_dataframe(train_output_df, pm25_loc_df)
target = (target - train_mean) / train_std

In [421]:
X = train.copy()
y = target.copy()

print(X.shape)
print(y.shape)

In [423]:
# feature 개수
num_features = X.shape[1]

In [424]:
X_train = train_df.copy()
X_val = val_df.copy()
y_train = target[:,:,:int(days*0.8),:]
y_val = target[:,:,int(days*0.8):,:]

print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(25, 13, 1460, 24)
(25, 13, 365, 24)
(25, 1, 1460, 24)
(25, 1, 365, 24)


In [425]:
X = X.reshape(25*1825, 24, num_features)
y = y.reshape(25*1825, 24)

X_train = X_train.reshape(25*1460, 24, num_features)
X_val = X_val.reshape(25*365, 24, num_features)
y_train = y_train.reshape(25*1460, 24)
y_val = y_val.reshape(25*365, 24)

## 3. Model train

In [428]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed

BATCH_SIZE = 256
EPOCHS = 10

'''
model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences = True, input_shape=(24, 14)))
#model.add(tf.keras.layers.RepeatVector(24))
#model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
model.summary()
'''

model = Sequential()
model.add(LSTM(16, activation='relu', return_sequences=True, input_shape=(24,num_features)))
model.add(LSTM(16, activation='relu'))
model.add(Dense(24))
model.compile(optimizer='adam', loss='mse')
model.summary()

"\nmodel = Sequential()\nmodel.add(LSTM(64, activation='relu', return_sequences = True, input_shape=(24, 14)))\n#model.add(tf.keras.layers.RepeatVector(24))\n#model.add(LSTM(32, activation='relu', return_sequences=True))\nmodel.add(TimeDistributed(Dense(1)))\nmodel.compile(optimizer='adam', loss='mse')\nmodel.summary()\n"

In [430]:
model_history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val))

Epoch 1/10
143/143 [==============================] - 23s 147ms/step - loss: 3.9556 - val_loss: 4.1482
Epoch 2/10
143/143 [==============================] - 21s 145ms/step - loss: 2.5912 - val_loss: 4.0952
Epoch 3/10
143/143 [==============================] - 21s 149ms/step - loss: 2.5743 - val_loss: 4.1221
Epoch 4/10
143/143 [==============================] - 21s 148ms/step - loss: 2.5629 - val_loss: 4.1093
Epoch 5/10
143/143 [==============================] - 21s 148ms/step - loss: 2.5529 - val_loss: 4.0995
Epoch 6/10
143/143 [==============================] - 21s 148ms/step - loss: 2.5463 - val_loss: 4.1713
Epoch 7/10
143/143 [==============================] - 21s 147ms/step - loss: 2.5334 - val_loss: 4.2389
Epoch 8/10
143/143 [==============================] - 21s 147ms/step - loss: 2.5236 - val_loss: 4.3224
Epoch 9/10
143/143 [==============================] - 21s 146ms/step - loss: 2.5182 - val_loss: 4.1547
Epoch 10/10
143/143 [==============================] - 21s 146ms/step - l

In [432]:
test_input = test_df.reshape(25*183, 24, num_features)
y_pred = model.predict(test_input)
y_pred = (y_pred*train_std) + train_mean
y_pred = y_pred.reshape(-1,1)

In [436]:
y_pred

array([[26.79734 ],
       [28.971733],
       [29.660122],
       ...,
       [24.264814],
       [24.453518],
       [24.68834 ]], dtype=float32)

In [437]:
# submission
submission['PM25'] = y_pred
submission.to_csv('/content/drive/MyDrive/air_submission.csv', index=False)
sub_path = '/content/drive/MyDrive/air_submission.csv'

In [438]:
submission

,id,time,loc_code,PM25
0,0,2020010200,111121,26.797340
1,0,2020010200,111123,28.971733
2,0,2020010200,111131,29.660122
3,0,2020010200,111141,29.163088
4,0,2020010200,111142,28.455427
...,...,...,...,...
109795,182,2020123123,111274,23.534342
109796,182,2020123123,111281,22.901804
109797,182,2020123123,111291,24.264814
109798,182,2020123123,111301,24.453518


In [439]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [440]:
w1 = WindowGenerator(input_width=24, label_width=24, shift=24)
w1

AttributeError: ignored

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
# Stack three slices, the length of the total window:
example_window = tf.stack([np.array(train_df[:w1.total_window_size]),
                           np.array(train_df[100:100+w1.total_window_size]),
                           np.array(train_df[200:200+w1.total_window_size])])


example_inputs, example_labels = w1.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'labels shape: {example_labels.shape}')

In [ ]:
def plot(self, model=None, plot_col='PM25', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [ ]:
w1.example = example_inputs, example_labels

In [ ]:
w1.plot()

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
# Each element is an (inputs, label) pair
w1.train.element_spec

In [ ]:
for example_inputs, example_labels in w1.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      callbacks=[early_stopping])
  return history

In [ ]:
single_step_window = WindowGenerator(
    # `WindowGenerator` returns all features as labels if you 
    # don't set the `label_columns` argument.
    input_width=1, label_width=1, shift=1)

wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=24, label_columns=['PM25'])

for example_inputs, example_labels in wide_window.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

In [ ]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', lstm_model(wide_window.example[0]).shape)

In [ ]:
history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate(wide_window.val)
performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)

In [ ]:
wide_window.test

In [ ]:
baseline = Baseline()
baseline.compile(loss=tf.losses.MeanSquaredError(),
                 metrics=[tf.metrics.MeanAbsoluteError()])

In [ ]:
val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(wide_window.val)
performance['Baseline'] = baseline.evaluate(wide_window.test, verbose=0)

In [ ]:
performance

In [ ]:
MAX_EPOCHS = 5

In [ ]:
%%time
wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=1)

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=num_features)
])

history = compile_and_fit(lstm_model, wide_window)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate( wide_window.val)
performance['LSTM'] = lstm_model.evaluate( wide_window.test, verbose=0)

print()

In [ ]:
y_pred = lstm_model.predict(wide_window.test)

In [ ]:
y_pred.shape

In [ ]:
y_pred

In [ ]:
submission.shape